In [1]:
import sys
sys.path.insert(0, '../../')
from time import sleep

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyplr import stlab

In [2]:
d = stlab.STLAB(username='admin', identity=1, password='83e47941d9e930f6')

STLAB device setup complete...


In [20]:
def interp_spectra(spectra):
    '''
    This function needs generalising.

    Parameters
    ----------
    spectra : TYPE
        DESCRIPTION.

    Returns
    -------
    intp_tbl : TYPE
        DESCRIPTION.

    '''
    #tbl = spectra.unstack(level=0)
    #tbl.columns = [val[1] for val in tbl.columns]
    
    intp_tbl = pd.DataFrame()
    for led, df in spectra.groupby(['led']):
        intensities = df.index.get_level_values('intensity')
        new_intensities = np.linspace(intensities.min(), intensities.max(), 4096)
        new_intensities = new_intensities.astype('int')
        df.reset_index(inplace=True, drop=True)
        #df.columns = range(0, df.shape[1])
        df.index = df.index * 63
        n = df.reindex(new_intensities).interpolate(method='linear')
        n['intensity'] = n.index
        n['led'] = led
        intp_tbl = intp_tbl.append(n)
    intp_tbl.set_index(['led','intensity'], inplace=True)
    return intp_tbl

In [56]:
cc = stlab.CalibrationContext('../data/corrected_oo_spectra.csv')
LED3_max = 1800 # comfortable maximal melanopsin stim
r_matches = {}
for led in range(10):
    match = cc.led_match(target_led=3, target_led_intensity=LED3_max, match_led=led, match_type='radiance')
    r_matches[led] = match

r_matches

{0: (0.004238363945272994, 2127),
 1: (0.0005316173873932684, 1692),
 2: (0.0028045785748886942, 1656),
 3: (0.0, 1800),
 4: (0.0008152973939665742, 1537),
 5: (0.00017104592243200045, 2113),
 6: (0.006184807102723511, 695),
 7: (0.006900740582779008, 831),
 8: (8.106384311723502e-05, 2795),
 9: (0.000973117048275185, 1431)}

In [52]:
from pyplr.CIE import get_CIE_1924_photopic_vl
vl = get_CIE_1924_photopic_vl(asdf=True)
cc.lkp.loc[(7,4095)].dot(vl.values)*683

array([18451.81050569])

In [57]:
cc.lkp

380       381       382      383       384       385  \
led intensity                                                              
0   0          0.000000  0.000000  0.000000  0.00000  0.000000  0.000000   
    1          0.000000  0.000000  0.000000  0.00000  0.000000  0.000000   
    2          0.000000  0.000000  0.000000  0.00000  0.000000  0.000000   
    3          0.000000  0.000000  0.000000  0.00000  0.000000  0.000000   
    4          0.000000  0.000000  0.000000  0.00000  0.000000  0.000000   
...                 ...       ...       ...      ...       ...       ...   
9   4091       0.009155  0.009682  0.008928  0.00919  0.007982  0.007132   
    4092       0.009155  0.009682  0.008928  0.00919  0.007982  0.007132   
    4093       0.009155  0.009682  0.008928  0.00919  0.007982  0.007132   
    4094       0.009155  0.009682  0.008928  0.00919  0.007982  0.007132   
    4095       0.009155  0.009682  0.008928  0.00919  0.007982  0.007132   

                    386       387       388       389  ...       771  \
led intensity                                          ...             
0   0          0.000000  0.000000  0.000000  0.000000  ...  0.000000   
    1          0.000000  0.000000  0.000000  0.000000  ...  0.000000   
    2          0.000000  0.000000  0.000000  0.000000  ...  0.000000   
    3          0.000000  0.000000  0.000000  0.000000  ...  0.000000   
    4          0.000000  0.000000  0.000000  0.000000  ...  0.000000   
...                 ...       ...       ...       ...  ...       ...   
9   4091       0.008048  0.006997  0.007932  0.009263  ...  0.005759   
    4092       0.008048  0.006997  0.007932  0.009263  ...  0.005759   
    4093       0.008048  0.006997  0.007932  0.009263  ...  0.005759   
    4094       0.008048  0.006997  0.007932  0.009263  ...  0.005759   
    4095       0.008048  0.006997  0.007932  0.009263  ...  0.005759   

                    772       773       774       775       776       777  \
led intensity                                                               
0   0          0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    1          0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    2          0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    3          0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    4          0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...                 ...       ...       ...       ...       ...       ...   
9   4091       0.006687  0.006306  0.005702  0.006548  0.006446  0.006078   
    4092       0.006687  0.006306  0.005702  0.006548  0.006446  0.006078   
    4093       0.006687  0.006306  0.005702  0.006548  0.006446  0.006078   
    4094       0.006687  0.006306  0.005702  0.006548  0.006446  0.006078   
    4095       0.006687  0.006306  0.005702  0.006548  0.006446  0.006078   

                    778       779       780  
led intensity                                
0   0          0.000000  0.000000  0.000000  
    1          0.000000  0.000000  0.000000  
    2          0.000000  0.000000  0.000000  
    3          0.000000  0.000000  0.000000  
    4          0.000000  0.000000  0.000000  
...                 ...       ...       ...  
9   4091       0.005479  0.006169  0.006359  
    4092       0.005479  0.006169  0.006359  
    4093       0.005479  0.006169  0.006359  
    4094       0.005479  0.006169  0.006359  
    4095       0.005479  0.006169  0.006359  

[40960 rows x 401 columns]

In [58]:
cc.data

380       381       382       383       384       385  \
led intensity                                                               
0   0          0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    65         0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    130        0.000000  0.000000  0.000000  0.000000  0.000000  0.000010   
    195        0.000131  0.000131  0.000115  0.000123  0.000117  0.000122   
    260        0.000412  0.000398  0.000386  0.000393  0.000366  0.000360   
...                 ...       ...       ...       ...       ...       ...   
9   3835       0.009495  0.009239  0.008684  0.008582  0.008071  0.007779   
    3900       0.008607  0.009418  0.008601  0.008611  0.007953  0.007391   
    3965       0.009032  0.008906  0.009078  0.009090  0.007391  0.007648   
    4030       0.008891  0.009872  0.008403  0.009253  0.007976  0.007476   
    4095       0.009155  0.009682  0.008928  0.009190  0.007982  0.007132   

                    386       387           388       389  ...       771  \
led intensity                                              ...             
0   0          0.000000  0.000000  0.000000e+00  0.000000  ...  0.000000   
    65         0.000000  0.000000  0.000000e+00  0.000000  ...  0.000000   
    130        0.000000  0.000009  7.505766e-07  0.000000  ...  0.000000   
    195        0.000113  0.000139  1.256129e-04  0.000138  ...  0.000068   
    260        0.000396  0.000351  3.645488e-04  0.000416  ...  0.000210   
...                 ...       ...           ...       ...  ...       ...   
9   3835       0.008634  0.007467  7.492949e-03  0.008368  ...  0.005477   
    3900       0.008173  0.007041  7.673470e-03  0.008338  ...  0.005882   
    3965       0.008266  0.007499  7.565773e-03  0.008577  ...  0.005885   
    4030       0.008297  0.006997  7.650152e-03  0.008423  ...  0.005905   
    4095       0.008048  0.006997  7.932088e-03  0.009263  ...  0.005759   

                    772       773       774       775       776       777  \
led intensity                                                               
0   0          0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    65         0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    130        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    195        0.000067  0.000061  0.000066  0.000049  0.000059  0.000061   
    260        0.000216  0.000208  0.000192  0.000205  0.000210  0.000177   
...                 ...       ...       ...       ...       ...       ...   
9   3835       0.006555  0.006369  0.005894  0.006261  0.006354  0.005772   
    3900       0.006751  0.006269  0.005853  0.006398  0.006484  0.005649   
    3965       0.006507  0.006038  0.005599  0.006427  0.006315  0.005767   
    4030       0.006736  0.006180  0.005642  0.006464  0.006382  0.006071   
    4095       0.006687  0.006306  0.005702  0.006548  0.006446  0.006078   

                    778       779       780  
led intensity                                
0   0          0.000000  0.000000  0.000000  
    65         0.000000  0.000000  0.000000  
    130        0.000000  0.000000  0.000000  
    195        0.000000  0.000060  0.000068  
    260        0.000130  0.000209  0.000228  
...                 ...       ...       ...  
9   3835       0.005046  0.006363  0.006376  
    3900       0.005427  0.006528  0.006225  
    3965       0.005288  0.005778  0.006553  
    4030       0.005596  0.005942  0.006588  
    4095       0.005479  0.006169  0.006359  

[640 rows x 401 columns]

In [4]:
d.set_spectrum_a([0,0,0,2000,0,0,0,0,0,0])

In [6]:
from scipy.signal import chirp

for led in r_matches.keys():

    # stimulus params
    videoname = 'chirp_{}'.format(led)
    Fs = 100
    mintensity = 0
    maxtensity = r_matches[led][1]

    # stimulus params
    background  = maxtensity/2
    amplitude   = maxtensity/2
    duration = 20
    n = 2000
    t = np.linspace(0, duration, n, endpoint=False)
    f0 = 0.01
    f1 = 1

    # define frequency modulation
    chp = chirp(t, f0, duration, f1, method='linear', phi=270)
    chp = stlab.modulate_intensity_amplitude(chp, background, amplitude)
    #fig, ax = plt.subplots(figsize=(12,4))
    #ax.plot(chp)
    #ax.set_ylabel("Intensity")
    #ax.set_xlabel("Time")

    # define amplitude modulation
    time  = stlab.get_time_vector(duration)
    sm = stlab.sinusoid_modulation(.5, duration, Fs)
    ivals = stlab.modulate_intensity_amplitude(sm, background, np.linspace(0,amplitude,duration*Fs))
    #fig, ax = plt.subplots(figsize=(12,4))
    #ax.plot(ivals)
    #ax.set_ylabel("Intensity")
    #ax.set_xlabel("Time");

    duration=60
    time = stlab.get_time_vector(duration)
    stim = np.zeros(duration*Fs)

    # assemble time course
    stim[0:100]     = maxtensity # 1s pulse
    stim[800:900]   = background # step
    stim[900:2900]  = chp  # chirp
    stim[2900:3100] = background
    stim[3100:5100] = ivals
    stim[5100:5300] = background
    stim = stim.astype('int')

    # plot
    #fig, ax = plt.subplots(figsize=(16,4))
    #ax.plot(stim)
    #ax.set_ylabel("Intensity")
    #ax.set_xlabel("Time")

    # make video file
    cols = stlab.get_video_cols()
    zs = np.zeros(duration*Fs)
    zs = zs.astype('int')
    data = [zs for x in range(10)]
    data[led] = stim

    #data = [stim for x in range(10)]
    data.insert(0, time)
    df = pd.DataFrame(data, cols).T
    stlab.make_video_file(df, videoname)

"chirp_0.dsf" saved in the current working directory.
"chirp_1.dsf" saved in the current working directory.
"chirp_2.dsf" saved in the current working directory.
"chirp_3.dsf" saved in the current working directory.
"chirp_4.dsf" saved in the current working directory.
"chirp_5.dsf" saved in the current working directory.
"chirp_6.dsf" saved in the current working directory.
"chirp_7.dsf" saved in the current working directory.
"chirp_8.dsf" saved in the current working directory.
"chirp_9.dsf" saved in the current working directory.


In [8]:
import glob
from time import sleep

for chp in glob.glob('chirp_*.dsf'):
    d.load_video_file(chp)
    d.play_video_file()
    sleep(60)

video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...


In [28]:
glob.glob('chirp_*.dsf')

['chirp_4.dsf',
 'chirp_5.dsf',
 'chirp_7.dsf',
 'chirp_6.dsf',
 'chirp_2.dsf',
 'chirp_3.dsf',
 'chirp_1.dsf',
 'chirp_0.dsf',
 'chirp_8.dsf',
 'chirp_9.dsf']